In [3]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

# 네이버 금융에서 제공하는 종목별 재무제표 크롤러 만들기

In [117]:
def ttt(xx):
    name = xx
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    return base_url

In [119]:
ttt('066571')

'https://finance.naver.com/item/coinfo.nhn?code=066571&target=finsum_more'

In [120]:
name = '011070'

In [121]:
base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'

In [122]:
base_url

'https://finance.naver.com/item/coinfo.nhn?code=011070&target=finsum_more'

In [6]:
browser  = Chrome()
browser.maximize_window()

In [123]:
browser.get(base_url)

In [124]:
browser.switch_to_frame(browser.find_element_by_id('coinfo_cp')) 
#frame구조 안으로 들어가기

In [125]:
#재무제표 "연간" 클릭하기
browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

In [126]:
html0 = browser.page_source

In [127]:
html1 = BeautifulSoup(html0,'html.parser')

In [146]:
title0 = html1.find('head').find('title').text

In [186]:
title0.split('-')[-1]

' 기업개요(LG이노텍)'

In [169]:
#( 와 ) 를 분기로 회사명만 뽑아낼 수 있는 방안은 혼자 고민해보기 정안되면 구글릉...
re.findall('(()+.([가-힣a-zA-Z0-9]).+())+',title0)

[('온라인기업정보 - 기업모니터 - 기업개요(LG이노텍)', '', '라', '')]

In [128]:
#find_all 명령어를 통해서 table태그의 class속성의 요소가 gHead01 all-width인 것들을 모두찾는다.
#이때 html2의 3번째 요소에 수집할 정보가 담겨져 있다.
html2 = html1.find_all('table',{'class':'gHead01 all-width'})

In [129]:
html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})

#### 날짜 긁어오기

In [130]:
thead0 = html22.find('thead')

In [131]:
tr0 = thead0.find_all('tr')[1]

In [132]:
th0 = tr0.find_all('th')

In [133]:
date = []
for i in range(len(th0)):
    date.append(''.join(re.findall('[0-9/]',th0[i].text)))

#### 본문데이터 및 columns 수집

In [134]:
tbody0 = html22.find('tbody')

In [135]:
tr0 = tbody0.find_all('tr')

In [136]:
col = []
for i in range(len(tr0)):
    
    if '\xa0' in tr0[i].find('th').text:
        tx = re.sub('\xa0','',tr0[i].find('th').text)
    else:
        tx = tr0[i].find('th').text
        
    col.append(tx)

In [137]:
td = []
for i in range(len(tr0)):
    td0 = tr0[i].find_all('td')
    td1 = []
    for j in range(len(td0)):
        if td0[j].text == '':
            td1.append('0')
        else:
            td1.append(td0[j].text)
            
    td.append(td1)

In [138]:
td2 = list(map(list,zip(*td)))

In [139]:
pd.DataFrame(td2,columns = col,index = date)

,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2014/12,"64,661","3,140","3,140","1,916","1,127","1,127",0,"44,288","27,323","16,965",...,161.05,"1,339.17","5,020",22.41,"71,689",1.57,250,0.22,5.25,"23,667,107"
2015/12,"61,381","2,237","2,237","1,221",951,951,0,"39,143","21,493","17,651",...,121.77,"1,399.15","4,018",24.52,"74,586",1.32,350,0.36,8.71,"23,667,107"
2016/12,"57,546","1,048","1,048",107,50,50,0,"43,237","25,452","17,785",...,143.11,"1,416.42",209,422.41,"75,154",1.18,250,0.28,119.45,"23,667,107"
2017/12,"76,414","2,965","2,965","2,387","1,748","1,748",0,"58,775","39,267","19,508",...,201.29,"1,584.48","7,385",19.50,"82,435",1.75,250,0.17,3.38,"23,667,107"
2018/12,"79,821","2,635","2,635","1,840","1,631","1,631",0,"57,561","36,378","21,182",...,171.74,"1,724.22","6,891",12.54,"89,511",0.97,300,0.35,4.35,"23,667,107"
2019/12,"80,962","2,781",0,"1,804","1,458","1,452",0,"62,679","40,071","22,607",...,177.25,0,"6,133",15.46,"95,607",0.99,292,0.31,4.76,0
2020/12,"89,176","3,541",0,"2,777","2,164","2,090",0,"64,875","40,234","24,641",...,163.28,0,"8,830",10.74,"104,364",0.91,308,0.32,3.49,0
2021/12,"93,929","3,981",0,"3,113","2,439","2,448",0,"67,863","40,743","27,122",...,150.22,0,"10,342",9.17,"114,610",0.83,344,0.36,3.33,0


# 네이버 종목별 재무제표 크롤러

In [189]:
browser  = Chrome()
browser.maximize_window()

In [190]:
def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    browser.get(base_url)
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    
    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    return pd.DataFrame(td2,columns = col,index = date)

In [191]:
#롯데 쇼핑의 재무제표
stock_crawler('023530')

 기업개요(롯데쇼핑)


,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2014/12,"280,996","11,884","11,884","10,780","6,157","5,266",891,"400,723","225,284","175,439",...,128.41,"10,333.68","16,724",15.43,"565,975",0.46,"2,000",0.78,11.22,"31,490,892"
2015/12,"291,277","8,537","8,537",-798,"-3,455","-3,831",376,"406,934","236,055","170,878",...,138.14,"10,037.45","-12,164",-18.07,"549,788",0.40,"2,000",0.91,-15.43,"31,490,892"
2016/12,"241,143","7,633","9,404","3,195","2,469","1,682",787,"419,159","246,519","172,640",...,142.79,"10,102.20","5,341",39.20,"551,720",0.38,"2,000",0.96,35.14,"31,490,892"
2017/12,"179,261","8,010","8,010","2,540",-206,"-1,364","1,158","279,485","145,971","133,514",...,109.33,"10,835.63","-4,437",-44.85,"450,420",0.44,"5,200",2.61,-107.15,"28,122,047"
2018/12,"178,208","5,970","5,970",-939,"-4,650","-5,449",0,"262,593","138,319","124,274",...,111.30,"10,297.93","-19,327",-10.92,"414,048",0.51,"5,200",2.46,-26.98,"28,288,755"
2019/12,"181,122","7,349",0,"4,517","3,389","3,005",0,"267,204","143,282","123,922",...,115.62,0,"10,624",14.97,"410,656",0.39,"4,400",2.77,41.42,0
2020/12,"183,823","7,936",0,"5,533","3,911","3,589",0,"269,561","142,860","126,700",...,112.75,0,"12,688",12.53,"418,658",0.38,"4,458",2.80,35.14,0
2021/12,"187,355","8,362",0,"6,264","4,424","3,952",0,"273,228","143,345","129,883",...,110.36,0,"13,970",11.38,"428,012",0.37,"4,483",2.82,32.09,0
